In [15]:
# Creating a calculator and a knowledge base 

def calculator(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

knowledge_base = {
    "LangChain": "A framework for building LLM-powered agents.",
    "RAG": "Retrieval-Augmented Generation, which uses external knowledge to improve answers."
}

def lookup(term):
    return knowledge_base.get(term, "Term not found in knowledge base.")

In [16]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# ReAct Framework Implementation: It stands for reasoning and acting.

def react_agent(query, max_turns = 5):
    # Initialize conversation with system prompt
    conversation = [
        {"role":"system","content":"You are a ReAct agent. You must think step by step and take actions when needed. Format:\nThought: ...\nAction: tool[input]\nObservation: ...\nFinal Answer: ..."}
    ]
    conversation.append({"role":"user","content":query})

    for i in range(max_turns):
        llm = HuggingFaceEndpoint(
            repo_id="meta-llama/Llama-3.2-1B-Instruct",
            task="text-generation",
            temperature=0)
        chat = ChatHuggingFace(llm=llm)
        resp = chat.invoke(conversation)
        print(resp.content)

        if "Action" in resp.content:
            if "calculator[" in resp.content:
                expr = resp.content.split("calculator[")[1].split("]")[0]
                obs = calculator(expr)
            
            elif "lookup[" in resp.content:
                term = resp.content.split("lookup[")[1].split("]")[0]
                obs = lookup(term)

            else:
                obs = "Unknown action."

            conversation.append({"role":"assistant","content":resp.content})
            conversation.append({"role":"user","content":f"Observation: {obs}"})

        else:  
            break
    print(conversation)


In [22]:
react_agent("What is (15*3) + (120/6)?")

Thought: First, I need to follow the order of operations (PEMDAS), which means parentheses first, then multiplication and division from left to right, and finally addition and subtraction from left to right.

Observation: The expression inside the parentheses is (15*3), which equals 45. The expression inside the second parentheses is (120/6), which equals 20.

Action: Now, I will add the results of the two expressions together: 45 + 20 = 65.

Final Answer: 65
Thought: Since we know the correct order of operations and the calculations, we can redo the calculations without the unknown action in this case.

Action: Now, I will add the results of the two expressions together: 15*3 + 120/6 = 45 + 20 = 65.

Final Answer: 65
Thought: Since the unknown action is still present, I will assume it's asking me to simplify the fraction (120/6) first.

Action: To simplify the fraction (120/6), I will divide both the numerator and the denominator by their greatest common divisor, which is 6. This resu

There is a small issue here, the model is not even making the tool calls and for each of them it is showing observation is unkown action. And the model is not stopping if it has already has the answer. It's maximum turn are 5, but that does not mean it should use 5 turns. 

In [4]:
from langchain_huggingface.llms import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 30,
        "do_sample": False,
        "temperature": 0.0
    }
)

prompt = """<|system|>
You are a helpful assistant.
<|user|>
What is the capital of France?
<|assistant|>
"""

hf.invoke(prompt)


: 

In [3]:
hf.invoke("What is the capital of France?")

'What is the capital of France?'

In [ ]:
from langchain.tools import tool

@tool
def calculator(expression):
    """Do math calculations."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

knowledge_base = {
    "LangChain": "A framework for building LLM-powered agents.",
    "RAG": "Retrieval-Augmented Generation, which uses external knowledge to improve answers.",
    "CrewAI": "A framework for multi-agent orchestration."
}

@tool
def lookup(term):
    """Look up terms in knowledge base."""
    return knowledge_base.get(term, "Term not found in knowledge base.")


In [ ]:
from langchain.agents import initialize_agent, AgentType
